# Create MBTiles from vector type data

This notebook transforms vector type data into `MBTiles` using tippecanoe.

## Table of Contents
### [Python libraries](#libraries)
### [Utils](#utils)
- **[create_mbtiles](#create_mbtiles)**

### [Read data](#carto)
- **[Indigenous Territories](#indigenous_territories)**
- **[Land Rights](#land_rights)**
- **[Major Mineral Deposits](#mineral_deposits)**

### [Create `MBTiles`](#create_mbtiles_2)
### [Show `MBTiles` in our localhost](#show_mbtiles)

<a id='libraries'></a>
### Python libraries

In [1]:
import json
import requests
import geopandas as gpd
import subprocess

<a id='utils'></a>
### Utils

<a id='create_mbtiles'></a>
**create_mbtiles**

In [2]:
def create_mbtiles(source_path, dest_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel"):
    """
    Use tippecanoe to create a MBTILE at dest_path from source_path.
    layer_name is used for the name of the layer in the MBTILE.
    Regex file path (/*.geojson) is supported for source_path.
    """
    cmd = f"tippecanoe -o {dest_path} -l {layer_name} {opts} {source_path}"
    print(f"Processing: {cmd}")
    r = subprocess.call(cmd, shell=True)
    if r == 0:
        print("Task created")
    else:
        print("Task failed")
    print("Finished processing")

<a id='carto'></a>
### Read data 

<a id='indigenous_territories'></a>
**[Indigenous Territories](https://native-land.ca/)**

In [ ]:
url = 'https://native-land.ca/coordinates/indigenousTerritories.json'

data = requests.get(url).json()

**Save data as `JSON`**

In [ ]:
with open('../data/mbtiles/Indigenous_Territories.json', 'w') as fp:
    json.dump(data, fp)

<a id='land_rights'></a>
**[Land Rights](http://data.globalforestwatch.org/datasets/e22b22b3525c46f3b35548daf6289903_1/data)**

In [16]:
data = gpd.read_file('../data/mbtiles/gfw_land_rights/gfw_land_rights.shp')
data['geometry'] = data['geometry'].apply(lambda x: x.simplify(0.0075))
data.head()

,country,name,legal_term,legal_reco,area_ha,source,last_updat,gfwid,st_area_sh,st_length_,geometry
0,CAN,ENGLISH RIVER HAULTAIN LAKE I.R. 192K,Indian Reserve,Y,201.288027174093,"Government of Canada, Natural Resources Canada...",2014,c531a195-09d5-449c-84fb-129d7bb77901,6.703042e+06,11842.772655,"POLYGON ((-11833624.371 7725300.176, -11832861..."
1,CAN,ELAK DASE I.R. 192A,Indian Reserve,Y,1397.67492142709,"Government of Canada, Natural Resources Canada...",2014,47f27ab3-f33b-407a-91dc-39667821c216,4.429801e+07,40679.063996,"POLYGON ((-11887246.473 7528355.931, -11887244..."
2,CAN,MIN-A-HE-QUO-SIS INDIAN RESERVE NO. 116A,Indian Reserve,Y,260.538624047784,"Government of Canada, Natural Resources Canada...",2014,0c445e83-c893-4fb3-9efb-efd0519a3df6,7.207503e+06,10749.287035,"POLYGON ((-12139665.105 6999456.605, -12138323..."
3,CAN,TSU TUE 196G,Indian Reserve,Y,42.7328901923613,"Government of Canada, Natural Resources Canada...",2014,349f8363-39ed-480b-bfd7-4228f5f3f4f1,1.698421e+06,5973.105602,"POLYGON ((-12307276.499 8389447.707, -12307276..."
4,CAN,WHITE FISH LAKE 128,Indian Reserve,Y,6156.88101888983,"Government of Canada, Natural Resources Canada...",2014,502a3edf-d25c-4121-863f-2a17dd1ab4d4,1.805401e+08,100897.806092,"POLYGON ((-12445854.141 7221022.378, -12445888..."


In [18]:
data = data.to_crs("EPSG:4326")
data.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

**Save data as `JSON`**

In [22]:
data.to_file('../data/mbtiles/Land_Rights.json', driver="GeoJSON")

<a id='mineral_deposits'></a>
**[Major Mineral Deposits](https://mrdata.usgs.gov/major-deposits/)**

In [8]:
data = gpd.read_file('../data/mbtiles/major_mineral_deposits/ofr20051294.shp')
data.head()

,REC_ID,DEP_NAME,COUNTRY,STATE,COMMODITY,DEP_TYPE,DETAIL,MODEL,DEP_MODEL,CITATION,CATEGORY,URL,geometry
0,325,Wando,Korean Peninsula,None,Pyrophyllite,Hydrothermal,None,None,None,"O'Driscoll, 1993",None,https://mrdata.usgs.gov/major-deposits/show-of...,POINT (126.68330 34.35000)
1,52,Georgina Basin,Australia,None,Phosphorous,Sedimentary,None,None,None,"Industrial Minerals, 1998",None,https://mrdata.usgs.gov/major-deposits/show-of...,POINT (139.96670 -21.88330)
2,1880,Wheal Jane,United Kingdom,None,Tin,Hydrothermal,None,None,None,None,None,https://mrdata.usgs.gov/major-deposits/show-of...,POINT (-5.20000 50.23333)
3,1979,Kaldygajtinskoe,Kazakhstan,None,Halite,Sedimentary,None,None,None,None,None,https://mrdata.usgs.gov/major-deposits/show-of...,POINT (53.60583 49.80639)
4,2548,Imiter,Morocco,None,Silver,Hydrothermal,None,None,None,None,None,https://mrdata.usgs.gov/major-deposits/show-of...,POINT (-5.83360 31.35000)


**Save data as `JSON`**

In [9]:
data.to_file('../data/mbtiles/Major_Mineral_Deposits.json', driver="GeoJSON")

<a id='create_mbtiles_2'></a>
### Create `MBTiles`

In [23]:
layers = {"Indigenous Territories": 'Indigenous_Territories',
         "Land Rights": 'Land_Rights',
         "Major Mineral Deposits": 'Major_Mineral_Deposits'}

for layer_name, file in layers.items():

    source_path = f"../data/mbtiles/{file}.json"
    dest_path = f"../data/mbtiles/{file}.mbtiles"
    create_mbtiles(source_path, dest_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel")

Processing: tippecanoe -o ../data/mbtiles/Land_Rights.mbtiles -l Land Rights -zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel ../data/mbtiles/Land_Rights.json
Task created
Finished processing


<a id='show_mbtiles'></a>
### Show `MBTiles` in our localhost

In [24]:
!mbview --port 9000 ../data/mbtiles/Land_Rights.mbtiles

Listening on http://localhost:9000
^C
